# pyIRF

Code adapted from the pyIRF example to LST-1 data

In [ ]:
import lstchain
print(f"lstchain version {lstchain.__version__}")
import pyirf
print(f"pyirf version {pyirf.__version__}")

In [ ]:
import matplotlib.pyplot as plt
import ctaplot
ctaplot.set_style('notebook')

In [ ]:
import tables
import numpy as np
import astropy.units as u
from astropy.table import QTable
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
import operator
from pathlib import Path
import numpy as np
from astropy import table
import astropy.units as u
from astropy.io import fits

from pyirf.binning import (
    create_bins_per_decade,
    add_overflow_bins,
    create_histogram_table,
)
from pyirf.cuts import calculate_percentile_cut, evaluate_binned_cut
from pyirf.sensitivity import calculate_sensitivity, estimate_background
from pyirf.utils import calculate_theta, calculate_source_fov_offset
from pyirf.benchmarks import energy_bias_resolution, angular_resolution
from pyirf.benchmarks.energy_bias_resolution import energy_resolution_absolute_68

from pyirf.spectral import (
    calculate_event_weights,
    PowerLaw,
    CRAB_HEGRA,
    IRFDOC_PROTON_SPECTRUM,
    IRFDOC_ELECTRON_SPECTRUM,
)
from pyirf.cut_optimization import optimize_gh_cut

from pyirf.irf import (
    effective_area_per_energy,
    energy_dispersion,
    psf_table,
    background_2d,
)

from pyirf.io import (
    create_aeff2d_hdu,
    create_psf_table_hdu,
    create_energy_dispersion_hdu,
    create_rad_max_hdu,
    create_background_2d_hdu,
)

from lstchain.io.io import read_mc_dl2_to_QTable

from lstchain.reco.utils import filter_events

In [ ]:
T_OBS = 50 * u.hour

# scaling between on and off region.
# Make off region 10 times larger than on region for better
# background statistics
ALPHA = 0.1

# Radius to use for calculating background rate
MAX_BG_RADIUS = 1 * u.deg
MAX_GH_CUT_EFFICIENCY = 0.9
GH_CUT_EFFICIENCY_STEP = 0.01

# gh cut used for first calculation of the binned theta cuts = initial proportion of gammas to keep
INITIAL_GH_CUT_EFFICENCY = 0.4

MIN_THETA_CUT = 0.1 * u.deg
MAX_THETA_CUT = 0.5 * u.deg

MIN_ENERGY = 20.0 * u.GeV
MAX_ENERGY = 20.05 * u.TeV

# same number of bins per decade than official CTA IRFs
N_BIN_PER_DECADE = 5



In [ ]:
ls /fefs/aswg/data/mc/DL2/20200629_prod5_trans_80/gamma/zenith_20deg/south_pointing/20210923_v0.7.5_prod5_trans_80_dynamic_cleaning/off0.4deg/

In [ ]:
gamma_file = '/fefs/aswg/data/mc/DL2/20200629_prod5_trans_80/gamma/zenith_20deg/south_pointing/20210923_v0.7.5_prod5_trans_80_dynamic_cleaning/off0.4deg/dl2_gamma_20deg_180deg_off0.4deg_20210923_v0.7.5_prod5_trans_80_dynamic_cleaning_testing.h5'
proton_file = '/fefs/aswg/data/mc/DL2/20200629_prod5_trans_80/proton/zenith_20deg/south_pointing/20210923_v0.7.5_prod5_trans_80_dynamic_cleaning/dl2_proton_20deg_180deg_20210923_v0.7.5_prod5_trans_80_dynamic_cleaning_testing.h5'
electron_file = '/fefs/aswg/data/mc/DL2/20200629_prod5_trans_80/electron/zenith_20deg/south_pointing/20210923_v0.7.5_prod5_trans_80_dynamic_cleaning/dl2_electron_20deg_180deg_20210923_v0.7.5_prod5_trans_80_dynamic_cleaning_testing.h5'

particles = {
    "gamma": {"file": gamma_file, "target_spectrum": CRAB_HEGRA},
    "proton": {"file": proton_file, "target_spectrum": IRFDOC_PROTON_SPECTRUM},
    "electron": {
        "file": electron_file,
        "target_spectrum": IRFDOC_ELECTRON_SPECTRUM,
    },
}

In [ ]:
filters = {
    'intensity': [10, np.inf],
}

## Data Loading

In [ ]:
for particle_type, p in particles.items():
    p["events"], p["simulation_info"] = read_mc_dl2_to_QTable(p["file"])
    p['events'] = filter_events(p['events'], filters)
    p["simulated_spectrum"] = PowerLaw.from_simulation(p["simulation_info"], T_OBS)
    p["events"]["weight"] = calculate_event_weights(
            p["events"]["true_energy"], p["target_spectrum"], p["simulated_spectrum"]
        )
    
    for prefix in ("true", "reco"):
            k = f"{prefix}_source_fov_offset"
            p["events"][k] = calculate_source_fov_offset(p["events"], prefix=prefix)

    
    
gammas = particles["gamma"]["events"]
# background table composed of both electrons and protons
background = table.vstack(
    [particles["proton"]["events"], particles["electron"]["events"]]
)

source_alt, source_az = gammas['true_alt'][0], gammas['true_az'][0]
for particle_type, p in particles.items():
    # calculate theta / distance between reco and assumed source position
    # we handle only ON observations here, so the assumed source pos is the pointing position
    p["events"]["theta"] = calculate_theta(p["events"], assumed_source_az=source_az, assumed_source_alt=source_alt)


In [ ]:
print(particles['gamma']['simulation_info'])

gammas[:5]

## First round

In [ ]:
hist_opt = dict(density=False, alpha=0.8, bins=40, log=True)
fig, ax = plt.subplots()
ax.hist(gammas["gh_score"], cumulative=True, **hist_opt, label='cumul')
ax.hist(gammas["gh_score"], cumulative=False, **hist_opt, color=plt.rcParams['axes.prop_cycle'].by_key()['color'][2])
ax.set_xlabel('GH score')
ax.legend()
plt.show()

In [ ]:
INITIAL_GH_CUT = np.quantile(gammas["gh_score"], (1 - INITIAL_GH_CUT_EFFICENCY))
theta_energy_bins = add_overflow_bins(create_bins_per_decade(MIN_ENERGY, MAX_ENERGY, N_BIN_PER_DECADE))

print(f"theta energy bins: {theta_energy_bins}")

# theta cut is 68 percent containment of the gammas
# for now with a fixed global, unoptimized score cut

mask_theta_cuts = gammas["gh_score"] >= INITIAL_GH_CUT
theta_cuts = calculate_percentile_cut(
    gammas["theta"][mask_theta_cuts],
    gammas["reco_energy"][mask_theta_cuts],
    bins=theta_energy_bins,
    min_value=MIN_THETA_CUT,
    fill_value=MAX_THETA_CUT,
    max_value=MAX_THETA_CUT,
    percentile=68,
)

theta_cuts

In [ ]:
from astropy.visualization import quantity_support

In [ ]:
plt.figure(figsize=(7,4))
with quantity_support():
    plt.errorbar(theta_cuts['center'], theta_cuts['cut'], xerr=(theta_cuts['center']-theta_cuts['low'], theta_cuts['high']-theta_cuts['center']), ls='--')
plt.xscale('log')
plt.ylabel('theta cut / deg')
plt.grid(True, which='both')
plt.show()

In [ ]:
sensitivity_bins = add_overflow_bins(create_bins_per_decade(MIN_ENERGY, MAX_ENERGY, bins_per_decade=N_BIN_PER_DECADE))
sensitivity_bins

In [ ]:
# Optimizing G/H separation cut for best sensitivity
gh_cut_efficiencies = np.arange(
    GH_CUT_EFFICIENCY_STEP,
    MAX_GH_CUT_EFFICIENCY + GH_CUT_EFFICIENCY_STEP / 2,
    GH_CUT_EFFICIENCY_STEP,
)

sensitivity_step_2, gh_cuts = optimize_gh_cut(
        gammas,
        background,
        reco_energy_bins=sensitivity_bins,
        gh_cut_efficiencies=gh_cut_efficiencies,
        op=operator.ge,
        theta_cuts=theta_cuts,
        alpha=ALPHA,
        background_radius=MAX_BG_RADIUS,
)

In [ ]:
sensitivity_step_2

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

ax = axes[0]

_,_,_,im = ax.hist2d(gammas['reco_energy'].to_value(u.TeV), gammas['gh_score'],
           bins=(np.logspace(-2, 1, 100), np.linspace(0, 1, 100)),
           norm=LogNorm()
          );
plt.colorbar(im, ax=ax)

ax.plot(gh_cuts['center'].to_value(u.TeV), gh_cuts['cut'], color='red', ls='--')
ax.fill_between(gh_cuts['center'].to_value(u.TeV), 0, gh_cuts['cut'], color='none', alpha=0.8, hatch="x", edgecolor='red', label='removed')
ax.set_title('Gammas events selection based on GH score')


### Exercise: do the same with the background

ax = axes[1]
ax.set_title('Background events selection based on GH score')


for ax in axes:
    ax.set_xscale('log')
    ax.set_xlabel('Energy/TeV')
    ax.set_ylabel('gh score')
    ax.legend()

plt.show()

In [ ]:
# now that we have the optimized gh cuts, we recalculate the theta
# cut as 68 percent containment on the events surviving these cuts.

for tab in (gammas, background):
    tab["selected_gh"] = evaluate_binned_cut(
        tab["gh_score"], tab["reco_energy"], gh_cuts, operator.ge
    )
    
theta_cuts_opt = calculate_percentile_cut(
        gammas[gammas["selected_gh"]]["theta"],
        gammas[gammas["selected_gh"]]["reco_energy"],
        theta_energy_bins,
        percentile=68,
        fill_value=MAX_THETA_CUT,
        max_value=MAX_THETA_CUT,
        min_value=MIN_THETA_CUT,
)

gammas["selected_theta"] = evaluate_binned_cut(
        gammas["theta"], gammas["reco_energy"], theta_cuts_opt, operator.le
    )

theta_cuts_opt

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 3))

ax = axes[0]

_,_,_,im = ax.hist2d(gammas['reco_energy'].to_value(u.TeV), gammas['theta'].to_value(u.deg),
           bins=(np.logspace(-2, 1, 100), np.linspace(0, 2, 100)),
           norm=LogNorm()
          );
plt.colorbar(im, ax=ax)

ax.plot(theta_cuts['center'].to_value(u.TeV), theta_cuts['cut'].to_value(u.deg), color='red', ls='--')
ax.fill_between(theta_cuts['center'].to_value(u.TeV), theta_cuts['cut'].to_value(u.deg), 4, color='none', alpha=0.8, hatch="x", edgecolor='red', label='removed')
ax.set_title('Gammas selection based on theta cut')


### Exercise: do the same with the background

ax = axes[1]

ax.set_title('Background events selection based on theta cut')


for ax in axes:
    ax.set_xscale('log')
    ax.set_xlabel('Energy/TeV')
    ax.set_ylabel('theta (deg)')
    ax.legend()

plt.show()

In [ ]:
gammas["selected"] = gammas["selected_theta"] & gammas["selected_gh"]

print(
    f"Total number of gammas: {len(gammas)}\n",
    f"After GH score selection: {100*np.count_nonzero(gammas['selected_gh'])/len(gammas):.2f}%\n",
    f"After theta selection: {100*np.count_nonzero(gammas['selected_theta'])/len(gammas):.2f}%\n",
    f"After both selection: {100*np.count_nonzero(gammas['selected'])/len(gammas):.2f}%\n",
)

In [ ]:
# calculate sensitivity
signal_hist = create_histogram_table(
    gammas[gammas["selected"]], bins=sensitivity_bins
)
background_hist = estimate_background(
    background[background["selected_gh"]],
    reco_energy_bins=sensitivity_bins,
    theta_cuts=theta_cuts_opt,
    alpha=ALPHA,
    background_radius=MAX_BG_RADIUS,
)
sensitivity = calculate_sensitivity(signal_hist, background_hist, alpha=ALPHA)

# scale relative sensitivity by Crab flux to get the flux sensitivity
spectrum = particles["gamma"]["target_spectrum"]
for s in (sensitivity_step_2, sensitivity):
    s["flux_sensitivity"] = s["relative_sensitivity"] * spectrum(s["reco_energy_center"])
    s["flux_sensitivity"] = s["flux_sensitivity"].to(1/(u.TeV * u.cm**2 * u.s))


sensitivity

## Calculating IRFs

In [ ]:

hdus = [
    fits.PrimaryHDU(),
    fits.BinTableHDU(sensitivity, name="SENSITIVITY"),
    fits.BinTableHDU(sensitivity_step_2, name="SENSITIVITY_STEP_2"),
    fits.BinTableHDU(theta_cuts, name="THETA_CUTS"),
    fits.BinTableHDU(theta_cuts_opt, name="THETA_CUTS_OPT"),
    fits.BinTableHDU(gh_cuts, name="GH_CUTS"),
]

masks = {
    "": gammas["selected"],
    "_NO_CUTS": slice(None),
    "_ONLY_GH": gammas["selected_gh"],
    "_ONLY_THETA": gammas["selected_theta"],
}

# binnings for the irfs
true_energy_bins = add_overflow_bins(
    create_bins_per_decade(MIN_ENERGY, MAX_ENERGY, N_BIN_PER_DECADE)
)
reco_energy_bins = add_overflow_bins(
    create_bins_per_decade(MIN_ENERGY, MAX_ENERGY, N_BIN_PER_DECADE)
)

fov_offset_bins = [0, 0.6] * u.deg
source_offset_bins = np.arange(0, 1 + 1e-4, 1e-3) * u.deg
energy_migration_bins = np.geomspace(0.2, 5, 200)

for label, mask in masks.items():
    effective_area = effective_area_per_energy(
        gammas[mask],
        particles["gamma"]["simulation_info"],
        true_energy_bins=true_energy_bins,
    )
    hdus.append(
        create_aeff2d_hdu(
            effective_area[..., np.newaxis],  # add one dimension for FOV offset
            true_energy_bins,
            fov_offset_bins,
            extname="EFFECTIVE_AREA" + label,
        )
    )
    edisp = energy_dispersion(
        gammas[mask],
        true_energy_bins=true_energy_bins,
        fov_offset_bins=fov_offset_bins,
        migration_bins=energy_migration_bins,
    )
    hdus.append(
        create_energy_dispersion_hdu(
            edisp,
            true_energy_bins=true_energy_bins,
            migration_bins=energy_migration_bins,
            fov_offset_bins=fov_offset_bins,
            extname="ENERGY_DISPERSION" + label,
        )
    )

bias_resolution = energy_bias_resolution(
    gammas[gammas["selected"]],
    true_energy_bins,
    resolution_function=energy_resolution_absolute_68,
)
ang_res = angular_resolution(gammas[gammas["selected_gh"]], true_energy_bins)
psf = psf_table(
    gammas[gammas["selected_gh"]],
    true_energy_bins,
    fov_offset_bins=fov_offset_bins,
    source_offset_bins=source_offset_bins,
)

background_rate = background_2d(
    background[background["selected_gh"]],
    reco_energy_bins,
    fov_offset_bins=np.arange(0, 11) * u.deg,
    t_obs=T_OBS,
)

hdus.append(
    create_background_2d_hdu(
        background_rate, reco_energy_bins, fov_offset_bins=np.arange(0, 11) * u.deg
    )
)
hdus.append(
    create_psf_table_hdu(psf, true_energy_bins, source_offset_bins, fov_offset_bins)
)
hdus.append(
    create_rad_max_hdu(
        theta_cuts_opt["cut"][:, np.newaxis], theta_energy_bins, fov_offset_bins
    )
)
hdus.append(fits.BinTableHDU(ang_res, name="ANGULAR_RESOLUTION"))
hdus.append(fits.BinTableHDU(bias_resolution, name="ENERGY_BIAS_RESOLUTION"))


In [ ]:
## Writing output file
outfile = "irf_sensitivity.fits.gz"
Path(outfile).parent.mkdir(exist_ok=True)
fits.HDUList(hdus).writeto(outfile, overwrite=True)

In [ ]:
from lstmcpipe.plots import plot_irfs

In [ ]:
plot_irfs.plot_effective_area_from_file(outfile, label='LST-1 MC')
ctaplot.plot_effective_area_cta_performance('north')
plt.show()

In [ ]:
plot_irfs.plot_angular_resolution_from_file(outfile, label='LST-1 MC')
ctaplot.plot_angular_resolution_cta_performance('north')
plt.show()

In [ ]:
plot_irfs.plot_energy_resolution_from_file(outfile, label='LST-1 MC')
ctaplot.plot_angular_resolution_cta_performance('north')
plt.show()

In [ ]:
plot_irfs.plot_background_rate_from_file(outfile, label='LST-1 MC')
ctaplot.plot_background_rate_magic()
plt.show()

In [ ]:
plot_irfs.plot_sensitivity_from_file(outfile, label='LST-1 MC')
ctaplot.plot_sensitivity_cta_performance('north')
ctaplot.plot_sensitivity_magic_performance()
plt.show()